# Практическая работа к Уроку№3 Парсинг данных. HTML, Beautiful Soap

### Задание 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.


### Задание 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты, то есть цифра вводится одна, а запрос проверяет оба поля)

In [1]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from pymongo import MongoClient

In [2]:
# Проверяем число или нет
def isint(s):
    try:
        int(s)
        return True
    except ValueError:
        return False


# Проверяем есть ли уже такая запись в базе
def check(s):
    for doc in hh_vacancy.find({'link': s}):
        return True

In [3]:
# Подготовим базу данных
client = MongoClient('127.0.0.1', 27017)

db = client['hh_vacancy']
hh_vacancy = db.hh_vacancy


In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
search = 'python'
params = {'page': 0}
url_site = f'https://hh.ru'
url = url_site + f'/search/vacancy?text={search}&salary=&no_magic=true&ored_clusters=true&items_on_page=20&excluded_text='


In [ ]:
session = requests.Session()
articles_list = []

while True:
    response =session.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    if not articles:
        break
    for article in articles:
        article_info = {}
        site = 'hh.ru'
        info = article.find('a', {'class': 'serp-item__title'})
        name = info.text
        link = info.get['href']
        salary = article.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if not salary:
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            salary = salary.getText().replace('\u202f', '')
            salary = re.split(r'\s|-', salary)
            if  salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1])
            elif salary[0] == 'от':
                salary_min = int(salary[1])
                salary_max = None
            else:
                salary_min = int(salary[0])
                salary_max = int(salary[2])
                salary_currency = salary[3]

        article_info['salary_min'] = salary_min
        article_info['salary_max'] = salary_max
        article_info['salary_currency'] = salary_currency
        article_info['site'] = site
        article_info['name'] = name
        article_info['link'] = link
        articles_list.append(article_info)           
    
    print(f"Обработана страница №{params['page']}")
    params['page'] += 1
    
pprint(articles_list)


In [ ]:
"""
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию. 
Добавить в решение со сбором вакансий(продуктов) функцию, которая будет добавлять 
только новые вакансии/продукты в вашу базу.
"""
 # Проверяем есть ли такая запись в базе
        if check(article_info['link']) is True:
            continue
        else:
            # Добавляем запись в базу
            hh_vacancy.insert_one(article_info)

    # Проверяем есть ли кнопка "дальше"
    if not button_next or not response.ok:
        break

    # Добавляем в параметры страницу
    page += 1

In [ ]:
"""
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы 
(необходимо анализировать оба поля зарплаты - минимальнную и максимульную). 
"""
# Проверяем есть ли null
def check(s):
    if s:
        return s
    else:
        return int('0')


salary_info = int(input('Введите желаемую зарплату: '))

for doc in hh_vacancy.find({'$or': [{'salary_currency': 'руб.'},
                                    {'salary_currency': 'USD'},
                                    ]
                            },
                           ):
    if doc['salary_currency'] == 'руб.':
        if check(doc['salary_min']) > salary_info or check(doc['salary_max']) > salary_info:
            pprint(doc)
    elif doc['salary_currency'] == 'USD':
        if (check(doc['salary_min']) * 80) > salary_info or (check(doc['salary_max']) * 80) > salary_info:
            pprint(doc)
    else:
        continue